In [5]:
# Install required packages (run this first)
# !pip install torch torchvision torchaudio
# !pip install pandas numpy scikit-learn matplotlib seaborn
# !pip install pycox torchtuples tqdm

import torch as tc
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import copy
import os
import warnings
warnings.filterwarnings('ignore')

# Set device for MacBook Pro M3
device = 'mps' if tc.backends.mps.is_available() else 'cpu'
print(f"Using device: {device}")

# Set random seeds for reproducibility
tc.manual_seed(42)
np.random.seed(42)
os.chdir('/Users/gabrielduarte/Documents/GitHub/clinical-trial-als-deepl')
print(os.getcwd())

Using device: mps
/Users/gabrielduarte/Documents/GitHub/clinical-trial-als-deepl


In [6]:
# Configuration for ALS Clinical Trials Analysis
config = {
    # Data params
    'core_file': 'db/core_trial_info.csv',
    'target_column': 'reached_phase_3_plus',
    'id_column': 'Trial.ID',
    
    # Model params (optimized for smaller dataset and M3 Mac)
    'hidden_depth_simple': 0,   
    'factor_hidden_nodes': 4,  # Reduced for smaller dataset
    'device': device,
    'batch_size': 32,  # Smaller batch size for M3
    'reduce_lr_epochs': [30, 30, 30],  # Fewer epochs
    'lr': 1e-3,
    'dropout': 0.3,
    'input_dropout': 0.3,
    'weight_decay': 1e-4,
    'splits': 3,  # 3-fold CV for smaller dataset
    'lrp_gamma': 0.01
}

print("Configuration loaded:")
for key, value in config.items():
    print(f"  {key}: {value}")

Configuration loaded:
  core_file: db/core_trial_info.csv
  target_column: reached_phase_3_plus
  id_column: Trial.ID
  hidden_depth_simple: 0
  factor_hidden_nodes: 4
  device: mps
  batch_size: 32
  reduce_lr_epochs: [30, 30, 30]
  lr: 0.001
  dropout: 0.3
  input_dropout: 0.3
  weight_decay: 0.0001
  splits: 3
  lrp_gamma: 0.01


In [7]:
def load_and_analyze_data(filepath):
    """Load and analyze the core trial data"""
    print("Loading ALS Clinical Trials data...")
    df = pd.read_csv(filepath)
    
    print(f"\nDataset Overview:")
    print(f"- Shape: {df.shape}")
    print(f"- Missing values: {df.isnull().sum().sum()} ({df.isnull().sum().sum()/df.size*100:.1f}%)")
    
    print(f"\nTarget Variable Distribution:")
    target_dist = df[config['target_column']].value_counts()
    print(target_dist)
    print(f"Success rate: {target_dist[1]/len(df)*100:.1f}%")
    
    print(f"\nTrial Phase Distribution:")
    print(df['Trial Phase'].value_counts())
    
    print(f"\nMissing values by column:")
    missing = df.isnull().sum()
    print(missing[missing > 0])
    
    return df

# Load the data
df = load_and_analyze_data(config['core_file'])

Loading ALS Clinical Trials data...

Dataset Overview:
- Shape: (962, 14)
- Missing values: 256 (1.9%)

Target Variable Distribution:
reached_phase_3_plus
0    743
1    219
Name: count, dtype: int64
Success rate: 22.8%

Trial Phase Distribution:
Trial Phase
Ii        337
I         296
Iii       177
Iv         82
Ii/Iii     42
Other      16
(N/A)      12
Name: count, dtype: int64

Missing values by column:
Study Keywords     171
Trial Objective      1
Study Design        84
dtype: int64


In [8]:
def preprocess_als_data(df, config):
    """Preprocess ALS clinical trials data for neural network"""
    
    # Columns to exclude from features
    exclude_cols = [
        config['target_column'], 
        config['id_column'], 
        'Protocol/Trial ID', 
        'Record URL', 
        'Trial Title',
        'Trial Objective',  # Too many unique values
        'Study Design',     # Too many unique values
        'MeSH Term'        # Too many unique values
    ]
    
    # Select feature columns
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    print(f"Using {len(feature_cols)} feature columns")
    
    # Prepare feature data
    X = df[feature_cols].copy()
    y = df[config['target_column']].copy()
    
    # Handle missing values
    for col in X.select_dtypes(include=['object']).columns:
        X[col] = X[col].fillna('Missing')
    
    # One-hot encode categorical variables
    X_encoded = pd.get_dummies(X, drop_first=True)
    print(f"After encoding: {X_encoded.shape[1]} features")
    
    # Convert to numpy and normalize
    X_array = X_encoded.values.astype(np.float32)
    y_array = y.values.astype(np.float32)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_array)
    
    # Create phase groups for stratification
    phase_groups = df['Trial Phase'].values
    
    return X_scaled, y_array, phase_groups, X_encoded.columns.tolist(), scaler

# Preprocess the data
X, y, phase_groups, feature_names, scaler = preprocess_als_data(df, config)
print(f"\nFinal dataset shape: X={X.shape}, y={y.shape}")
print(f"Feature names: {feature_names[:5]}...")  # Show first 5 features

Using 6 feature columns
After encoding: 379 features

Final dataset shape: X=(962, 379), y=(962,)
Feature names: ['Disease_Autoimmune/Inflammation: Asthma; CNS: Amyotrophic Lateral Sclerosis; CNS: Attention Deficit Hyperactive Disorder', 'Disease_Autoimmune/Inflammation: Psoriasis; Autoimmune/Inflammation: Rheumatoid Arthritis; Autoimmune/Inflammation: Ulcerative Colitis; Cardiovascular: Peripheral Arterial Disease; CNS: Amyotrophic Lateral Sclerosis', 'Disease_CNS: Alcohol Dependence; CNS: Amyotrophic Lateral Sclerosis; CNS: Anxiety; CNS: Multiple Sclerosis', 'Disease_CNS: Alcohol Dependence; CNS: Amyotrophic Lateral Sclerosis; CNS: Multiple Sclerosis; Oncology: CNS, Glioblastoma; Oncology: Colorectal; Oncology: Supportive Care', "Disease_CNS: Alzheimer's Disease"]...


In [9]:
class LRP_Linear(nn.Module):
    def __init__(self, inp, outp, gamma=0.01, eps=1e-5):
        super(LRP_Linear, self).__init__()
        self.A_dict = {}
        self.linear = nn.Linear(inp, outp)
        nn.init.xavier_uniform_(self.linear.weight, gain=nn.init.calculate_gain('relu'))
        self.gamma = tc.tensor(gamma)
        self.eps = tc.tensor(eps)
        self.iteration = None

    def forward(self, x):
        if not self.training:
            self.A_dict[self.iteration] = x.clone()
        return self.linear(x)

    def relprop(self, R):
        device = next(self.parameters()).device
        A = self.A_dict[self.iteration].clone()
        A, self.eps = A.to(device), self.eps.to(device)

        Ap = A.clamp(min=0).detach().data.requires_grad_(True)
        Am = A.clamp(max=0).detach().data.requires_grad_(True)

        zpp = self.newlayer(1).forward(Ap)  
        zmm = self.newlayer(-1, no_bias=True).forward(Am) 
        zmp = self.newlayer(1, no_bias=True).forward(Am) 
        zpm = self.newlayer(-1).forward(Ap) 

        with tc.no_grad():
            Y = self.forward(A).data

        sp = ((Y > 0).float() * R / (zpp + zmm + self.eps * ((zpp + zmm == 0).float() + tc.sign(zpp + zmm)))).data
        sm = ((Y < 0).float() * R / (zmp + zpm + self.eps * ((zmp + zpm == 0).float() + tc.sign(zmp + zpm)))).data

        (zpp * sp).sum().backward()
        cpp = Ap.grad
        Ap.grad = None
        Ap.requires_grad_(True)

        (zpm * sm).sum().backward()
        cpm = Ap.grad
        Ap.grad = None
        Ap.requires_grad_(True)

        (zmp * sm).sum().backward()
        cmp = Am.grad
        Am.grad = None
        Am.requires_grad_(True)

        (zmm * sp).sum().backward()
        cmm = Am.grad
        Am.grad = None
        Am.requires_grad_(True)

        R_1 = (Ap * cpp).data
        R_2 = (Ap * cpm).data
        R_3 = (Am * cmp).data
        R_4 = (Am * cmm).data

        return R_1 + R_2 + R_3 + R_4

    def newlayer(self, sign, no_bias=False):
        if sign == 1:
            rho = lambda p: p + self.gamma * p.clamp(min=0)
        else:
            rho = lambda p: p + self.gamma * p.clamp(max=0)

        layer_new = copy.deepcopy(self.linear)
        try:
            layer_new.weight = nn.Parameter(rho(self.linear.weight))
        except AttributeError:
            pass

        try:
            layer_new.bias = nn.Parameter(self.linear.bias * 0 if no_bias else rho(self.linear.bias))
        except AttributeError:
            pass

        return layer_new

class LRP_ReLU(nn.Module):
    def __init__(self):
        super(LRP_ReLU, self).__init__()
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(x)

    def relprop(self, R):
        return R

class LRP_DropOut(nn.Module):
    def __init__(self, p):
        super(LRP_DropOut, self).__init__()
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        return self.dropout(x)

    def relprop(self, R):
        return R

class ALS_Model(nn.Module):
    def __init__(self, n_features, config):
        super(ALS_Model, self).__init__()
        self.classname = 'ALS Clinical Trials Model'
        
        inp = n_features
        hidden = int(config['factor_hidden_nodes'] * n_features)
        outp = 1
        
        self.layers = nn.Sequential(
            LRP_DropOut(p=config['input_dropout']), 
            LRP_Linear(inp, hidden, gamma=config['lrp_gamma']), 
            LRP_ReLU()
        )
        
        # Add hidden layers if specified
        for i in range(config['hidden_depth_simple']):
            self.layers.add_module(f'dropout_{i}', LRP_DropOut(p=config['dropout']))
            self.layers.add_module(f'linear_{i}', LRP_Linear(hidden, hidden, gamma=config['lrp_gamma']))
            self.layers.add_module(f'relu_{i}', LRP_ReLU())
        
        self.layers.add_module('dropout_final', LRP_DropOut(p=config['dropout']))
        self.layers.add_module('linear_final', LRP_Linear(hidden, outp, gamma=config['lrp_gamma']))
        
        # Count parameters
        nparams = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f'{self.classname} contains {nparams:,} trainable parameters')

    def forward(self, x):
        return self.layers.forward(x)

    def relprop(self, R):
        assert not self.training, 'relprop does not work during training time'
        for module in self.layers[::-1]:
            R = module.relprop(R)
        return R

print("Neural network classes defined ✓")

Neural network classes defined ✓


In [13]:
import torchtuples as tt
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

def train_model(X_train, y_train, X_val, y_val, model, config):
    """Train the ALS model using Cox proportional hazards"""
    
    # Convert to tensors
    X_train_tensor = tc.tensor(X_train).float().to(config['device'])
    X_val_tensor = tc.tensor(X_val).float().to(config['device'])
    
    # Create survival data (using target as both duration and event)
    train_durations = tc.tensor(y_train).float()
    train_events = tc.tensor(y_train).bool()
    val_durations = tc.tensor(y_val).float()
    val_events = tc.tensor(y_val).bool()
    
    # Prepare data for pycox
    train_data = (X_train_tensor, (train_durations, train_events))
    val_data = (X_val_tensor, (val_durations, val_events))
    
    # Create CoxPH model
    optimizer = tt.optim.Adam(config['lr'], weight_decay=config['weight_decay'])
    coxph_model = CoxPH(model, optimizer, device=config['device'])
    
    # Training with learning rate reduction
    all_logs = []
    for exp, training_epochs in enumerate(config['reduce_lr_epochs']):
        print(f'Training for {training_epochs} epochs with lr {config["lr"]*10**(-exp):.6f}')
        coxph_model.optimizer.set_lr(config['lr']*10**(-exp))
        
        callbacks = [
            tt.callbacks.EarlyStopping(patience=10),
            tt.callbacks.ClipGradNorm(model, max_norm=1.0)
        ]
        
        log = coxph_model.fit(
            X_train_tensor, (train_durations, train_events),
            batch_size=config['batch_size'], 
            epochs=training_epochs, 
            callbacks=callbacks,
            val_data=val_data, 
            val_batch_size=config['batch_size'], 
            verbose=True
        )
        all_logs.append(log)
    
    return coxph_model, all_logs

def evaluate_model(coxph_model, X_test, y_test):
    """Evaluate the trained model"""
    X_test_tensor = tc.tensor(X_test).float().to(config['device'])
    test_durations = tc.tensor(y_test).float()
    test_events = tc.tensor(y_test).bool()
    
    # Compute baseline hazards and predictions
    _ = coxph_model.compute_baseline_hazards()
    surv_pred = coxph_model.predict_surv_df(X_test_tensor)
    
    # Compute evaluation metrics
    ev = EvalSurv(surv_pred, test_durations.numpy(), test_events.numpy(), censor_surv='km')
    concordance = ev.concordance_td()
    
    time_grid = np.linspace(test_durations.min().item(), test_durations.max().item(), 10)
    integrated_brier_score = ev.integrated_brier_score(time_grid)
    
    return concordance, integrated_brier_score, surv_pred

print("Training functions defined ✓")

Training functions defined ✓


In [14]:
from sklearn.model_selection import StratifiedKFold

def run_cross_validation(X, y, phase_groups, config):
    """Run stratified k-fold cross-validation"""
    
    # Create stratified folds
    skf = StratifiedKFold(n_splits=config['splits'], shuffle=True, random_state=42)
    
    results = []
    models = []
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, phase_groups)):
        print(f"\n{'='*50}")
        print(f"FOLD {fold + 1}/{config['splits']}")
        print(f"{'='*50}")
        
        # Split data
        X_train_fold, X_test_fold = X[train_idx], X[test_idx]
        y_train_fold, y_test_fold = y[train_idx], y[test_idx]
        
        # Further split training data for validation
        X_train, X_val, y_train, y_val = train_test_split(
            X_train_fold, y_train_fold, test_size=0.2, random_state=42, 
            stratify=phase_groups[train_idx]
        )
        
        print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test_fold)}")
        print(f"Test target distribution: {np.bincount(y_test_fold.astype(int))}")
        
        # Create and train model
        model = ALS_Model(X.shape[1], config).to(config['device'])
        coxph_model, logs = train_model(X_train, y_train, X_val, y_val, model, config)
        
        # Evaluate model
        concordance, brier_score, surv_pred = evaluate_model(coxph_model, X_test_fold, y_test_fold)
        
        print(f"\nFold {fold + 1} Results:")
        print(f"Concordance: {concordance:.4f}")
        print(f"Integrated Brier Score: {brier_score:.4f}")
        
        results.append({
            'fold': fold + 1,
            'concordance': concordance,
            'brier_score': brier_score,
            'n_test': len(X_test_fold),
            'n_positive': int(y_test_fold.sum())
        })
        
        models.append({
            'fold': fold + 1,
            'model': model,
            'coxph_model': coxph_model,
            'test_idx': test_idx,
            'logs': logs
        })
    
    return results, models

# Run cross-validation
print("Starting cross-validation...")
cv_results, trained_models = run_cross_validation(X, y, phase_groups, config)

# Display overall results
results_df = pd.DataFrame(cv_results)
print(f"\n{'='*60}")
print("CROSS-VALIDATION SUMMARY")
print(f"{'='*60}")
print(results_df)
print(f"\nMean Concordance: {results_df['concordance'].mean():.4f} ± {results_df['concordance'].std():.4f}")
print(f"Mean Brier Score: {results_df['brier_score'].mean():.4f} ± {results_df['brier_score'].std():.4f}")

Starting cross-validation...

FOLD 1/3
Train: 512, Val: 129, Test: 321
Test target distribution: [252  69]
ALS Clinical Trials Model contains 577,597 trainable parameters
Training for 30 epochs with lr 0.001000
0:	[5s / 5s],		train_loss: 2.0568
1:	[0s / 6s],		train_loss: 1.8059
2:	[0s / 6s],		train_loss: 1.7009
3:	[0s / 6s],		train_loss: 2.2675
4:	[0s / 6s],		train_loss: 1.9945
5:	[0s / 6s],		train_loss: 1.7346
6:	[0s / 6s],		train_loss: 1.8518
7:	[0s / 6s],		train_loss: 1.7443
8:	[0s / 7s],		train_loss: 1.7317
9:	[0s / 7s],		train_loss: 1.9512
Training for 30 epochs with lr 0.000100
10:	[0s / 0s],		train_loss: 1.9861
11:	[0s / 0s],		train_loss: 1.7099
12:	[0s / 0s],		train_loss: 2.2090
13:	[0s / 0s],		train_loss: 1.7207
14:	[0s / 0s],		train_loss: 1.5372
15:	[0s / 0s],		train_loss: 1.4967
16:	[0s / 0s],		train_loss: 1.4629
17:	[0s / 1s],		train_loss: 1.2888
18:	[0s / 1s],		train_loss: 1.3660
19:	[0s / 1s],		train_loss: 1.4003
Training for 30 epochs with lr 0.000010
20:	[0s / 0s],		tra

RuntimeError: Numpy is not available

In [ ]:
def compute_lrp_explanations(model, X_test, test_idx, feature_names):
    """Compute LRP explanations for test samples"""
    
    model.eval()
    X_test_tensor = tc.tensor(X_test).float().to(config['device'])
    
    # Forward pass to get predictions
    with tc.no_grad():
        predictions = model(X_test_tensor)
    
    # Compute LRP explanations
    model.eval()  # Ensure model is in eval mode for LRP
    explanations = []
    
    for i in range(len(X_test)):
        # Set iteration for LRP
        for module in model.modules():
            if hasattr(module, 'iteration'):
                module.iteration = i
        
        # Single sample prediction and explanation
        sample = X_test_tensor[i:i+1]
        pred = model(sample)
        
        # Compute LRP
        lrp_scores = model.relprop(pred)
        explanations.append(lrp_scores.cpu().numpy().flatten())
    
    explanations = np.array(explanations)
    
    # Create explanation DataFrame
    explanation_df = pd.DataFrame(explanations, columns=feature_names)
    explanation_df['prediction'] = predictions.cpu().numpy().flatten()
    explanation_df['sample_idx'] = test_idx
    
    return explanation_df

def analyze_feature_importance(explanation_df, feature_names, top_k=10):
    """Analyze feature importance from LRP explanations"""
    
    # Calculate mean absolute importance for each feature
    feature_importance = explanation_df[feature_names].abs().mean().sort_values(ascending=False)
    
    print(f"Top {top_k} Most Important Features:")
    print("-" * 50)
    for i, (feature, importance) in enumerate(feature_importance.head(top_k).items()):
        print(f"{i+1:2d}. {feature:<30} {importance:.6f}")
    
    return feature_importance

print("LRP analysis functions defined ✓")

In [ ]:
# Generate LRP explanations for the best performing fold
best_fold = cv_results[np.argmax([r['concordance'] for r in cv_results])]
best_model_info = trained_models[best_fold['fold'] - 1]

print(f"Analyzing LRP explanations for best fold (Fold {best_fold['fold']})...")
print(f"Best concordance: {best_fold['concordance']:.4f}")

# Get test data for best fold
test_idx = best_model_info['test_idx']
X_test_best = X[test_idx]
y_test_best = y[test_idx]
model_best = best_model_info['model']

# Compute LRP explanations
explanation_df = compute_lrp_explanations(model_best, X_test_best, test_idx, feature_names)

# Analyze feature importance
feature_importance = analyze_feature_importance(explanation_df, feature_names, top_k=15)

print(f"\nGenerated explanations for {len(explanation_df)} test samples")

In [ ]:
def plot_results(cv_results, feature_importance, explanation_df):
    """Create visualizations of results"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Cross-validation results
    ax1 = axes[0, 0]
    results_df = pd.DataFrame(cv_results)
    ax1.bar(range(1, len(results_df) + 1), results_df['concordance'])
    ax1.axhline(y=results_df['concordance'].mean(), color='r', linestyle='--', 
                label=f'Mean: {results_df["concordance"].mean():.3f}')
    ax1.set_xlabel('Fold')
    ax1.set_ylabel('Concordance Index')
    ax1.set_title('Cross-Validation Performance')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Feature importance
    ax2 = axes[0, 1]
    top_features = feature_importance.head(10)
    ax2.barh(range(len(top_features)), top_features.values)
    ax2.set_yticks(range(len(top_features)))
    ax2.set_yticklabels([f.replace('_', ' ')[:20] + '...' if len(f) > 20 else f.replace('_', ' ') 
                        for f in top_features.index])
    ax2.set_xlabel('Mean Absolute LRP Score')
    ax2.set_title('Top 10 Most Important Features')
    ax2.grid(True, alpha=0.3)
    
    # 3. Prediction distribution
    ax3 = axes[1, 0]
    ax3.hist(explanation_df['prediction'], bins=20, alpha=0.7, edgecolor='black')
    ax3.axvline(x=explanation_df['prediction'].mean(), color='r', linestyle='--', 
                label=f'Mean: {explanation_df["prediction"].mean():.3f}')
    ax3.set_xlabel('Model Prediction')
    ax3.set_ylabel('Frequency')
    ax3.set_title('Distribution of Model Predictions')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. LRP score distribution for top feature
    ax4 = axes[1, 1]
    top_feature = feature_importance.index[0]
    lrp_scores = explanation_df[top_feature]
    ax4.hist(lrp_scores, bins=20, alpha=0.7, edgecolor='black')
    ax4.axvline(x=lrp_scores.mean(), color='r', linestyle='--', 
                label=f'Mean: {lrp_scores.mean():.4f}')
    ax4.set_xlabel('LRP Score')
    ax4.set_ylabel('Frequency')
    ax4.set_title(f'LRP Score Distribution: {top_feature.replace("_", " ")}')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Create visualizations
plot_results(cv_results, feature_importance, explanation_df)

# Summary statistics
print(f"\n{'='*60}")
print("FINAL ANALYSIS SUMMARY")
print(f"{'='*60}")
print(f"Dataset: {X.shape[0]} ALS clinical trials with {X.shape[1]} features")
print(f"Target: Phase 3+ progression ({y.sum():.0f}/{len(y)} = {y.mean()*100:.1f}% success rate)")
print(f"Model Performance: {results_df['concordance'].mean():.4f} ± {results_df['concordance'].std():.4f} concordance")
print(f"Top predictive feature: {feature_importance.index[0]}")
print(f"Number of important features (>0.001): {(feature_importance > 0.001).sum()}")

In [ ]:
def analyze_top_features(explanation_df, feature_names, df, top_k=5):
    """Detailed analysis of top contributing features"""
    
    feature_importance = explanation_df[feature_names].abs().mean().sort_values(ascending=False)
    
    print("DETAILED FEATURE ANALYSIS")
    print("=" * 60)
    
    for i, feature in enumerate(feature_importance.head(top_k).index):
        print(f"\n{i+1}. {feature}")
        print("-" * 50)
        
        # LRP statistics
        lrp_scores = explanation_df[feature]
        print(f"LRP Score Statistics:")
        print(f"  Mean: {lrp_scores.mean():.6f}")
        print(f"  Std:  {lrp_scores.std():.6f}")
        print(f"  Range: [{lrp_scores.min():.6f}, {lrp_scores.max():.6f}]")
        
        # Feature interpretation
        if feature.endswith('_1.0') or feature.endswith('_True'):
            base_feature = feature.replace('_1.0', '').replace('_True', '')
            print(f"  Type: Binary indicator for '{base_feature}'")
        elif any(phase in feature for phase in ['_I', '_Ii', '_Iii', '_Iv']):
            print(f"  Type: Trial phase indicator")
        elif 'Status_' in feature:
            print(f"  Type: Trial status indicator")
        else:
            print(f"  Type: Feature encoding")
        
        # Correlation with target
        if len(lrp_scores) > 1:
            # Get corresponding actual outcomes for test samples
            test_outcomes = y[explanation_df['sample_idx']]
            correlation = np.corrcoef(lrp_scores, test_outcomes)[0, 1]
            print(f"  Correlation with outcome: {correlation:.4f}")

# Run detailed analysis
analyze_top_features(explanation_df, feature_names, df)

In [ ]:
# Save results for future analysis
def save_results():
    """Save analysis results"""
    
    # Create results directory
    os.makedirs('als_results', exist_ok=True)
    
    # Save cross-validation results
    pd.DataFrame(cv_results).to_csv('als_results/cv_results.csv', index=False)
    
    # Save feature importance
    feature_importance.to_csv('als_results/feature_importance.csv')
    
    # Save LRP explanations
    explanation_df.to_csv('als_results/lrp_explanations.csv', index=False)
    
    # Save model configuration
    with open('als_results/config.txt', 'w') as f:
        for key, value in config.items():
            f.write(f"{key}: {value}\n")
    
    print("Results saved to 'als_results/' directory")

# Uncomment to save results
# save_results()

print("\n🎉 Analysis Complete!")
print("You can now explore the LRP explanations to understand which")
print("trial characteristics contribute most to Phase 3+ progression.")